# Data Cleaning

## Introduction

This notebook goes through the steps taken with the data collected in order to get cleaned organized data in two standard text formats. The notebook will contain the nexts steps.

1. **Cleaning the data -** I will use text pre-procesing techniques to get the dta into shape.
2. **Organizing the data -** I'l organize the data into a way that is easy to input into other algoithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of texts
2. **Document-Term Matrix** - words counts in matrix format

### Problem Statement

My goal is to look look a the latest headlines of the main newspapers in Perú and note simmilarities and differences.

In [1]:
import demoji
import json
import numpy as np
import os
import pandas as pd
import pickle as pk
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import re
import requests
import string

from dotenv import load_dotenv
from itertools import product
from plotly.subplots import make_subplots
from tqdm import tqdm
from urllib.parse import urlencode

Since most of the data we are dealing with is text data, I'm goig to be using some common text pre-processing techniques.

For that I'm going to follow the MVP __(Minimum Viable Product)__ approach. For that the main resource I'll be using is a talk from PyOhio by [Alice Zhao](https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb). The cleaning steps I'll be taking are.

**Removing tweets that are outside of the scope**

* Tweets corresponding to the cover page announcement
* Tweets corresponding to caricature of the day
* Tweets corresponding to the horoscope
* ...

**Common data cleaning steps on all text:**

* Make text all lowercase
* Remove punctuation
* Remove numerical values
* Remove common non.sensical text (\n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**

* Stemming/lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos

In [2]:
load_dotenv()

BASE_DIR = os.environ.get("BASE_DIR")
BEARER_TOKEN = os.environ.get("BEARER_TOKEN")

In [3]:
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 25)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format",  "{:,.2f}".format)

pio.templates.default = "plotly_white"
pio.kaleido.scope.default_scale = 2

gruvbox_colors = ["#458588", "#FABD2F", "#B8BB26", "#CC241D", "#B16286", "#8EC07C", "#FE8019"]

## Data Loading

In [4]:
with open(f'{BASE_DIR}/data/raw/newspapers_id.json', 'r') as read_file:
    newspapers_id = json.load(read_file)

In [5]:
newspaper_df_list = []
WEEKS = [35, 36, 37]

for newspaper, week in product(newspapers_id, WEEKS):

    with open(f'{BASE_DIR}/data/raw/2022w{week}_data_{newspaper}.json', 'r') as read_file:
        json_file = json.load(read_file)

    json_data = json_file["data"]

    newspaper_df = pd.json_normalize(json_data)
    newspaper_df["newspaper"] = newspaper

    newspaper_df_list.append(newspaper_df)

In [6]:
data_raw = pd.concat(newspaper_df_list)

In [7]:
data_raw["created_at"] = pd.to_datetime(data_raw["created_at"], infer_datetime_format=True)

In [8]:
data_raw.columns = data_raw.columns.str.removeprefix("public_metrics.")

In [9]:
data_raw.head()

,created_at,possibly_sensitive,id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids
0,2022-08-28 23:57:24+00:00,False,1564039479391838209,Venezuela y Colombia retoman relaciones diplomáticas rotas hace tres años https://t.co/L6uVA6LcEE,0,0,6,1,NaN,elcomercio_peru,NaN
1,2022-08-28 23:49:59+00:00,False,1564037610393280512,“Me dijeron que estaba llevando vergüenza a la universidad”: la profesora obligada a renunciar por postear fotos en bikini https://t.co/zAe98GI7W2,0,0,5,1,NaN,elcomercio_peru,NaN
2,2022-08-28 23:29:00+00:00,False,1564032331706470401,AMLO afirma que familias ya aceptaron plan de rescate de 10 mineros https://t.co/dG3VJXWgNa,0,0,2,0,NaN,elcomercio_peru,NaN
3,2022-08-28 23:14:11+00:00,False,1564028601053347843,Zelensky: los ocupantes rusos sentirán las consecuencias de “futuras acciones” https://t.co/mNJTLz0SS7,6,7,18,1,NaN,elcomercio_peru,NaN
4,2022-08-28 23:09:07+00:00,False,1564027328157683713,Essalud: realizan con éxito operativo de donación de órganos para salvar vida de siete pacientes en espera https://t.co/3sDo7q9Nuu,1,0,11,0,NaN,elcomercio_peru,NaN


In [10]:
data_raw.tail()

,created_at,possibly_sensitive,id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids
20,2022-09-06 02:26:00+00:00,False,1566975975648366601,"""Dice Antauro Humala que él mandaría fusilar a varios expresidentes, incluyendo o, mejor dicho, empezando con su hermano. ¿Qué está fumando Antauro?"". - #Hildebrandt ahora en vivo en su podcast de los lunes a las 9 p.m. en https://t.co/ofhuiX1sZu",34,25,192,1,NaN,ensustrece,[1566975975648366601]
21,2022-09-06 02:09:05+00:00,False,1566971720187236352,RT @idiotas_para: “Un analfabeto funcional fundador de 3 universidades … esto es el Peru…” \nPodcast de Cesar Hildebrandt @ensustrece,252,0,0,0,"[{'type': 'retweeted', 'id': '1566970877408952320'}]",ensustrece,[1566971720187236352]
22,2022-09-05 23:23:11+00:00,False,1566929969460174848,Columna de @palidofuego111. https://t.co/DSL8Ly490T,11,4,40,0,"[{'type': 'quoted', 'id': '1566193986867453952'}]",ensustrece,[1566929969460174848]
23,2022-09-05 23:21:08+00:00,False,1566929453946748931,No necesita suscripción. El podcast es de acceso libre. Va en vivo cada lunes a las 9 p.m. en https://t.co/ofhuiX1sZu https://t.co/VD2VkMxy3c,30,7,114,0,"[{'type': 'quoted', 'id': '1566842263287873538'}]",ensustrece,[1566929453946748931]
24,2022-09-05 23:20:17+00:00,False,1566929240725114882,"RT @unchasqui: Hace unos días, César Hildebrandt de @ensustrece la llamó ""natacha"". Hoy, un audio revela cómo la ""natachean"". En serio, Lad…",492,0,0,0,"[{'type': 'retweeted', 'id': '1565778940614115330'}]",ensustrece,[1566929240725114882]


In [11]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34633 entries, 0 to 24
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   created_at              34633 non-null  datetime64[ns, UTC]
 1   possibly_sensitive      34633 non-null  bool               
 2   id                      34633 non-null  object             
 3   text                    34633 non-null  object             
 4   retweet_count           34633 non-null  int64              
 5   reply_count             34633 non-null  int64              
 6   like_count              34633 non-null  int64              
 7   quote_count             34633 non-null  int64              
 8   referenced_tweets       3484 non-null   object             
 9   newspaper               34633 non-null  object             
 10  edit_history_tweet_ids  639 non-null    object             
dtypes: bool(1), datetime64[ns, UTC](1), int64(4)

## First look on data

From the first looks, as well a the look on the head and tail I get to see how many of the variables are categorical and how many are numerical. Also, I only found missing values in the referenced tweets field. So, first for the numerical value I'm going to take a look of some metrics before the EDA.

**Numerical variables:**

1. `created_at`: Timestamp of the tweet
2. `public_metrics.retweet_count`: Number of times a tweet was retweeted
3. `public_metrics.like_count`: Number of likes a tweet has
4. `public_metrics.quote_count`: Number of times a tweet was quoted

**Categorical variables**

1. `id`: Unique identifier of tweet
2. `positively_sensitive`: Boolean variable of whether a tweet might contain sensitive information
3. `text`: Actual text of the tweet
4. `referenced_tweet`: Whether this tweet is a retweet or a quoted tweet
5. `newspaper`: Twitter handle if the newspaper the tweet belongs to

In [12]:
from matplotlib.pyplot import title


fig = px.histogram(
    data_raw,
    x="created_at",
    facet_col="newspaper",
    facet_col_wrap=2,
    color_discrete_sequence=gruvbox_colors,
    title="Number of tweets per newspaper",
    height=1600,
    width=1200
)

fig.update_traces(xbins_size="D1")

fig.show()

In [13]:
data_stats = pd.DataFrame()

data_stats["raw_tweet_count"] = data_raw["newspaper"].value_counts()

data_stats = data_stats.merge(data_raw.loc[data_raw["referenced_tweets"].notna(), "newspaper"].value_counts(), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"newspaper": "raw_referenced_tweet_count"}, inplace=True)

data_stats = data_stats.merge(data_raw.loc[data_raw["possibly_sensitive"] == True, "newspaper"].value_counts(), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"newspaper": "raw_possibly_sensitive_count"}, inplace=True)

data_stats = data_stats.merge(data_raw.groupby("newspaper").sum(numeric_only=True).drop("possibly_sensitive", axis=1), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"retweet_count": "raw_retweet_count", "reply_count": "raw_reply_count", "like_count": "raw_like_count", "quote_count": "raw_quote_count"}, inplace=True)

In [14]:
data_raw[data_raw["possibly_sensitive"] == True]

,created_at,possibly_sensitive,id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids
478,2022-09-09 02:28:12+00:00,True,1568063694348713985,Día Internacional de Prevención del Suicidio: Conozca las maneras adecuadas de evitarlo https://t.co/C78pTfnxwe,1,0,3,0,NaN,peru21noticias,NaN
845,2022-09-07 20:11:30+00:00,True,1567606507507777541,#VIDEO | Lima: Tenga cuidado con la táctica fraudulenta denominada ‘cambiazo’\n\nhttps://t.co/9wslNCg3Or,0,1,1,0,NaN,Gestionpe,NaN
851,2022-09-07 19:54:06+00:00,True,1567602127735853062,Lima: Tenga cuidado con la táctica fraudulenta denominada ‘cambiazo’ https://t.co/9wslNCg3Or,1,0,4,0,NaN,Gestionpe,NaN
674,2022-09-07 00:55:38+00:00,True,1567315623734018053,"Video donde Karla Tarazona solo sonríe para las fotos, se viraliza https://t.co/9q48ai3qy5",1,0,2,0,NaN,diarioojo,NaN


In [15]:
data_stats["raw_reference_to_tweets_ratio"] = data_stats["raw_referenced_tweet_count"]/data_stats["raw_tweet_count"]
data_stats["raw_sensitive_to_tweets_ratio"] = data_stats["raw_possibly_sensitive_count"]/data_stats["raw_tweet_count"]
data_stats["raw_retweet_to_tweets_ratio"] = data_stats["raw_retweet_count"]/data_stats["raw_tweet_count"]
data_stats["raw_reply_to_tweets_ratio"] = data_stats["raw_reply_count"]/data_stats["raw_tweet_count"]
data_stats["raw_like_to_tweets_ratio"] = data_stats["raw_like_count"]/data_stats["raw_tweet_count"]
data_stats["raw_quote_to_tweets_ratio"] = data_stats["raw_quote_count"]/data_stats["raw_tweet_count"]

In [16]:
data_stats

,raw_tweet_count,raw_referenced_tweet_count,raw_possibly_sensitive_count,raw_retweet_count,raw_reply_count,raw_like_count,raw_quote_count,raw_reference_to_tweets_ratio,raw_sensitive_to_tweets_ratio,raw_retweet_to_tweets_ratio,raw_reply_to_tweets_ratio,raw_like_to_tweets_ratio,raw_quote_to_tweets_ratio
peru21noticias,5394,195.00,1.00,43143,26942,135405,4228,0.04,0.00,8.00,4.99,25.10,0.78
Gestionpe,5037,1.00,2.00,6814,5886,24502,934,0.00,0.00,1.35,1.17,4.86,0.19
larepublica_pe,4874,"2,848.00",NaN,72389,11773,63155,2413,0.58,NaN,14.85,2.42,12.96,0.50
tromepe,4841,NaN,NaN,4749,11803,28007,1065,NaN,NaN,0.98,2.44,5.79,0.22
diarioojo,3669,NaN,1.00,1542,3895,12845,415,NaN,0.00,0.42,1.06,3.50,0.11
diariocorreo,3339,6.00,NaN,10188,9372,30041,1701,0.00,NaN,3.05,2.81,9.00,0.51
elcomercio_peru,3015,146.00,NaN,10103,3901,20841,821,0.05,NaN,3.35,1.29,6.91,0.27
ExpresoPeru,1674,1.00,NaN,53645,42974,173380,5517,0.00,NaN,32.05,25.67,103.57,3.30
DiarioElPeruano,1484,16.00,NaN,4295,2213,15998,509,0.01,NaN,2.89,1.49,10.78,0.34
larazon_pe,667,NaN,NaN,1781,266,3396,97,NaN,NaN,2.67,0.40,5.09,0.15


## Removing non relevant tweets

In [17]:
data = data_raw

data.drop(data[data["text"].str.contains('horóscopo diario', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo hoy', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo y tarot', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('horóscopo', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('Buenos días', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('caricatura de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('las caricaturas de', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('portada impresa', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('portada de hoy', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('en portada', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('trome gol', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('no te pierdas las chiquitas de hoy', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data[data["text"].str.contains('esta es la portada', flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data.loc[data["text"].str.contains("Aquí la portada del", flags=re.IGNORECASE, regex=True)].index, inplace=True)
data.drop(data.loc[data["text"].str.contains("yapaza", flags=re.IGNORECASE, regex=True)].index, inplace=True)

In [18]:
data_stats = data_stats.merge(data["newspaper"].value_counts(), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"newspaper": "clean_tweet_count"}, inplace=True)

data_stats = data_stats.merge(data.loc[data["referenced_tweets"].notna(), "newspaper"].value_counts(), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"newspaper": "clean_referenced_tweet_count"}, inplace=True)

data_stats = data_stats.merge(data.loc[data["possibly_sensitive"] == True, "newspaper"].value_counts(), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"newspaper": "clean_possibly_sensitive_count"}, inplace=True)

data_stats = data_stats.merge(data.groupby("newspaper").sum(numeric_only=True).drop("possibly_sensitive", axis=1), how="left", left_index=True, right_index=True)
data_stats.rename(columns={"retweet_count": "clean_retweet_count", "reply_count": "clean_reply_count", "like_count": "clean_like_count", "quote_count": "clean_quote_count"}, inplace=True)

In [19]:
data_stats["clean_reference_to_tweets_ratio"] = data_stats["clean_referenced_tweet_count"]/data_stats["clean_tweet_count"]
data_stats["clean_sensitive_to_tweets_ratio"] = data_stats["clean_possibly_sensitive_count"]/data_stats["clean_tweet_count"]
data_stats["clean_retweet_to_tweets_ratio"] = data_stats["clean_retweet_count"]/data_stats["clean_tweet_count"]
data_stats["clean_reply_to_tweets_ratio"] = data_stats["clean_reply_count"]/data_stats["clean_tweet_count"]
data_stats["clean_like_to_tweets_ratio"] = data_stats["clean_like_count"]/data_stats["clean_tweet_count"]
data_stats["clean_quote_to_tweets_ratio"] = data_stats["clean_quote_count"]/data_stats["clean_tweet_count"]

In [20]:
data_stats

,raw_tweet_count,raw_referenced_tweet_count,raw_possibly_sensitive_count,raw_retweet_count,raw_reply_count,raw_like_count,raw_quote_count,raw_reference_to_tweets_ratio,raw_sensitive_to_tweets_ratio,raw_retweet_to_tweets_ratio,...,clean_retweet_count,clean_reply_count,clean_like_count,clean_quote_count,clean_reference_to_tweets_ratio,clean_sensitive_to_tweets_ratio,clean_retweet_to_tweets_ratio,clean_reply_to_tweets_ratio,clean_like_to_tweets_ratio,clean_quote_to_tweets_ratio
peru21noticias,5394,195.00,1.00,43143,26942,135405,4228,0.04,0.00,8.00,...,31857,20360,101248,3303,0.04,0.00,7.14,4.57,22.71,0.74
Gestionpe,5037,1.00,2.00,6814,5886,24502,934,0.00,0.00,1.35,...,5374,5048,19631,761,0.00,0.00,1.29,1.21,4.72,0.18
larepublica_pe,4874,"2,848.00",NaN,72389,11773,63155,2413,0.58,NaN,14.85,...,59225,8553,48008,1842,0.59,NaN,14.93,2.16,12.10,0.46
tromepe,4841,NaN,NaN,4749,11803,28007,1065,NaN,NaN,0.98,...,3677,9409,22266,913,NaN,NaN,0.94,2.40,5.67,0.23
diarioojo,3669,NaN,1.00,1542,3895,12845,415,NaN,0.00,0.42,...,1147,2958,9833,337,NaN,NaN,0.40,1.03,3.43,0.12
diariocorreo,3339,6.00,NaN,10188,9372,30041,1701,0.00,NaN,3.05,...,7884,7397,23380,1305,0.00,NaN,3.05,2.86,9.03,0.50
elcomercio_peru,3015,146.00,NaN,10103,3901,20841,821,0.05,NaN,3.35,...,7574,2649,14582,562,0.05,NaN,3.29,1.15,6.33,0.24
ExpresoPeru,1674,1.00,NaN,53645,42974,173380,5517,0.00,NaN,32.05,...,33641,29025,113908,3733,0.00,NaN,28.06,24.21,95.00,3.11
DiarioElPeruano,1484,16.00,NaN,4295,2213,15998,509,0.01,NaN,2.89,...,3242,1590,11367,329,0.01,NaN,3.08,1.51,10.78,0.31
larazon_pe,667,NaN,NaN,1781,266,3396,97,NaN,NaN,2.67,...,272,43,667,30,NaN,NaN,0.63,0.10,1.55,0.07


### Extracting hashtags and mentions

From each of the tweets, I'll see if there are any hashtags or mentions

## Text cleaning and tokenization

When it comes to text processing, and specially for tweets, there are some common text patterns that do not add any meaning to the message being conveyed. For example: links, hashtags and tags.

In [27]:
data["hashtags"] = data.apply(lambda x: re.findall(r"#(\w+)", x["text"]), axis=1)
data["mentions"] = data.apply(lambda x: re.findall(r"@(\w+)", x["text"]), axis=1)

In [32]:
def clean_text_first_pass(text):
    """Get rid of other punctuation and non-sensical text identified.

    Args:
        text (string): text to be processed.
    """
    text = text.lower()
    text = re.sub("http[s]?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[‘’“”…«»►¿¡|│]', '', text)
    text = re.sub("\w*\d\w*", "", text)
    text = re.sub('\n', ' ', text)

    return text

first_pass = lambda x: clean_text_first_pass(x)

In [33]:
data["text_clean"] = data.text.apply(first_pass)
data["text_clean"]

0                                          más de  vehículos usan gnv en el país según infogas httpstcopztjxmgsjm
1              ecuador videos registran los efectos del sismo de magnitud  que causó pánico y daños en guayaquil 
2                                                                 adiós a desiderio blanco al maestro con cariño 
3                                                                      el perú vive en la tensión de dos fuerzas 
4                                   mvcs descarta que haya suspendido el programa techo propio httpstcoocmehugxbw
                                                          ...                                                    
734     vocal dirimente augusto ruidías farfán zanjó discordia de sus colegas de la sala constitucional lee aquí 
735                                                               según encuesta de datum internacional lee aquí 
736                                                               fondo a repartir super

In [34]:
data[data["possibly_sensitive"] == True]

,possibly_sensitive,created_at,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,newspaper,text_clean


In [35]:
data.rename(lambda x: x.replace('public_metrics.', ''), axis='columns', inplace=True)
data.head()

,possibly_sensitive,created_at,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,False,2022-07-14 23:59:57+00:00,"Más de 324.000 vehículos usan GNV en el país, según Infogas https://t.co/PztJXMGSJM",1547732663062081539,2,1,7,0,NaN,elcomercio_peru,más de vehículos usan gnv en el país según infogas httpstcopztjxmgsjm
1,False,2022-07-14 23:54:53+00:00,"Ecuador: videos registran los efectos del sismo de magnitud 6,1 que causó pánico y daños en Guayaquil https://t.co/vr6OEFri2L",1547731391546216454,6,0,14,0,NaN,elcomercio_peru,ecuador videos registran los efectos del sismo de magnitud que causó pánico y daños en guayaquil
2,False,2022-07-14 23:49:47+00:00,"Adiós a Desiderio Blanco: al maestro, con cariño https://t.co/A53I6JVtOX",1547730104922451969,1,0,9,0,NaN,elcomercio_peru,adiós a desiderio blanco al maestro con cariño
3,False,2022-07-14 23:44:39+00:00,“El Perú vive en la tensión de dos fuerzas” https://t.co/W2pgc4FMIT,1547728815090069505,0,3,12,1,NaN,elcomercio_peru,el perú vive en la tensión de dos fuerzas
4,False,2022-07-14 23:39:35+00:00,MVCS descarta que haya suspendido el programa Techo Propio https://t.co/OCMEHuGXBw,1547727539564449795,0,1,7,0,NaN,elcomercio_peru,mvcs descarta que haya suspendido el programa techo propio httpstcoocmehugxbw


From checking the resulting text I found that there are some tweets that contain emojis that haven't been removed. For that I will use the `demoji` package.

In [37]:
replace_emojis = lambda x: demoji.replace(x, "")

In [38]:
data["text_clean"] = data["text_clean"].apply(replace_emojis)
data.head()

,possibly_sensitive,created_at,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,False,2022-07-14 23:59:57+00:00,"Más de 324.000 vehículos usan GNV en el país, según Infogas https://t.co/PztJXMGSJM",1547732663062081539,2,1,7,0,NaN,elcomercio_peru,más de vehículos usan gnv en el país según infogas httpstcopztjxmgsjm
1,False,2022-07-14 23:54:53+00:00,"Ecuador: videos registran los efectos del sismo de magnitud 6,1 que causó pánico y daños en Guayaquil https://t.co/vr6OEFri2L",1547731391546216454,6,0,14,0,NaN,elcomercio_peru,ecuador videos registran los efectos del sismo de magnitud que causó pánico y daños en guayaquil
2,False,2022-07-14 23:49:47+00:00,"Adiós a Desiderio Blanco: al maestro, con cariño https://t.co/A53I6JVtOX",1547730104922451969,1,0,9,0,NaN,elcomercio_peru,adiós a desiderio blanco al maestro con cariño
3,False,2022-07-14 23:44:39+00:00,“El Perú vive en la tensión de dos fuerzas” https://t.co/W2pgc4FMIT,1547728815090069505,0,3,12,1,NaN,elcomercio_peru,el perú vive en la tensión de dos fuerzas
4,False,2022-07-14 23:39:35+00:00,MVCS descarta que haya suspendido el programa Techo Propio https://t.co/OCMEHuGXBw,1547727539564449795,0,1,7,0,NaN,elcomercio_peru,mvcs descarta que haya suspendido el programa techo propio httpstcoocmehugxbw


Also, form redoing the analysis, and looking at the tweeter feeds from many of the newspapers I found that there are some patterns of writing that do not add to the content, like calls to action, that interact with the audience, but do not add any significance to the headline. I'll be checking at the twitter feed for such patterns and add them during cleaning. I'm not adding them as stopwords because these calls to action in specific are groups of words.

In [39]:
def clean_text_second_pass(text):
    """Get rid of other punctuation and non-sensical text identified.

    Args:
        text (string): text to be processed.
    """
    text = re.sub('click aquí', '', text)
    text = re.sub('lee aquí el blog de', '', text)
    text = re.sub('vía gestionpe', '', text)
    text = re.sub('entrevista exclusiva', '', text)
    text = re.sub('en vivo', '', text)
    text = re.sub('entérate más aquí', '', text)
    text = re.sub('lee la columna de', '', text)
    text = re.sub('lee y comenta', '', text)
    text = re.sub('lea hoy la columna de', '', text)
    text = re.sub('escrito por', '', text)
    text = re.sub('una nota de', '', text)
    text = re.sub('aquí la nota', '', text)
    text = re.sub('nota completa aquí', '', text)
    text = re.sub('lee más', '', text)
    text = re.sub('lee aquí', '', text)

    text = re.sub('  ', ' ', text)

    return text

second_pass = lambda x: clean_text_second_pass(x)

In [40]:
data["text_clean"] = data.text_clean.apply(second_pass)
data.head()

,possibly_sensitive,created_at,text,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,text_clean
0,False,2022-07-14 23:59:57+00:00,"Más de 324.000 vehículos usan GNV en el país, según Infogas https://t.co/PztJXMGSJM",1547732663062081539,2,1,7,0,NaN,elcomercio_peru,más de vehículos usan gnv en el país según infogas
1,False,2022-07-14 23:54:53+00:00,"Ecuador: videos registran los efectos del sismo de magnitud 6,1 que causó pánico y daños en Guayaquil https://t.co/vr6OEFri2L",1547731391546216454,6,0,14,0,NaN,elcomercio_peru,ecuador videos registran los efectos del sismo de magnitud que causó pánico daños en guayaquil
2,False,2022-07-14 23:49:47+00:00,"Adiós a Desiderio Blanco: al maestro, con cariño https://t.co/A53I6JVtOX",1547730104922451969,1,0,9,0,NaN,elcomercio_peru,adiós desiderio blanco al maestro con cariño
3,False,2022-07-14 23:44:39+00:00,“El Perú vive en la tensión de dos fuerzas” https://t.co/W2pgc4FMIT,1547728815090069505,0,3,12,1,NaN,elcomercio_peru,el perú vive en la tensión de dos fuerzas
4,False,2022-07-14 23:39:35+00:00,MVCS descarta que haya suspendido el programa Techo Propio https://t.co/OCMEHuGXBw,1547727539564449795,0,1,7,0,NaN,elcomercio_peru,mvcs descarta que haya suspendido el programa techo propio


In [41]:
data.to_pickle(f"{BASE_DIR}/data/interim/data_clean.pkl")

## Organizing the data

Now I need to get the data in both of the standar text formats:

1. **Corpus -** a collection of text
2. **Document-Term matrix -** word counts in matrix format

In the case of the tweets, I will start by combining all the clean texts and in the case of *Document-term matrix* tokenising the result.

In [31]:
df_clean = pd.read_pickle(f"{BASE_DIR}/data/interim/data_clean.pkl")

### Corpus

To create a corpus we will collapse all the existing rows and create another dataframe with the name of the newspaper and the tweets for analysis.

In [32]:
data_corpus = df_clean.groupby("newspaper")["text_clean"].apply(' '.join).reset_index()
data_corpus.head(15)

,newspaper,text_clean
0,DiarioElPeruano,conoce las actividades virtuales que se desarrollarán por el aniversario de lima será transmitido por facebook youtube ministra de defensa nuria esparch resalta la presencia cada vez mayor de mujeres en misiones de paz ops alerta que la variante del se expande en latinoamérica crisis política...
1,ExpresoPeru,giulliana loza keiko fujimori está realizando visitas al norte del país con la debida autorización judicial médicos intensivistas sostienen que lima metropolitana debió ser calificada como una región con un nivel de alerta muy alto partido entre aston villa vs everton fue aplazado por casos de...
2,Gestionpe,depósitos en dólares en soles qué convendrá más este año sinovac defiende su vacuna tras datos decepcionantes de ensayo en brasil línea del metro de lima establece nuevo horario ante ampliación del toque de queda en alemania temen que variante británica del covid pueda multiplicar por diez la...
3,diariocorreo,san borja minsa lanzan farmacia con servicio delivery para población vulnerable médicos de essalud extirpan tumor de un kilo salvan niño arequipeño de años video hombre de años murió después de cobrar su pensión en un banco de huancayo jne declara infundada apelación del apra para inscripci...
4,diarioojo,octogenario muere después de cobrar su pensión en un banco de junín luciana fuster celebra su cumpleaños en yate de marc anthony el invitado de honor video mujer acusada de asesinar su esposo él se apuñaló solo video gigi mitre enfrenta son tentación hay que ser un poquito más conscientes res...
5,elcomercio_peru,keiko fujimori me llena de ilusión energía haber empezado caminar para llegar todo nuestro país línea del metro de lima nuevo horario ante ampliación del toque de queda la policía los militares de eeuu se enfrentan un enemigos internos autos antiguos en las calles limeñas sporting cristal of...
6,larazon_pe,la premier violeta bermúdez dijo que se empezará una campaña informativa sobre la importancia de la vacunación ya que muchas personas tienen temor no creen el exministro de salud lamentó que la titular del minsa no pueda inducir al presidente estrategias mucho más claras juez del primer juzgad...
7,larepublica_pe,rt deporteslr sportboys anunció que seis miembros del plantel dieron positivo través de un comunicado rosados informaron rt deporteslr el puma carranza se refirió la marcha de hohberg sporting cristal el histórico capitán de la habló acerca del te onu el mundo se dirige hacia un calentamiento c...
8,peru21noticias,se registra mamá oso de anteojos su cría paseando por las áreas de machu picchu en cusco video arturo vidal se puso emotivo para despedir reinaldo rueda de la selección de chile george forsyth rechaza dinero de publicidad electoral para que estado lo use en el sistema de salud lambayeque gere...
9,tromepe,no le teme los nuevos sabores lambayeque fiscalía interviene jefe de comisaría incahuasi otros cuatro efectivos por presuntos sobornos el club estudiantil arrancó los trabajos en su sede pensando en la temporada alex valera jorge murrugarra los nuevos fichajes dijeron presente en la sesión go...


In [33]:
data_corpus = data_corpus.set_index('newspaper')

In [34]:
data_corpus.to_pickle(f'{BASE_DIR}/data/processed/corpus.pkl')

### Document-Term Matrix

From the corpus constructed in the step above I'll proceed to tokenizethe text to use with further techniques. For that I'll use scikit-learn's `CountVectorizer`, where every row represents a document and each column is a different row.

I'll also remove stop words.

In [4]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [10]:
def normalize_text(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop | t.is_space]

    return words

normalize = lambda x: normalize_text(x)

In [9]:
data_dtm = pd.read_pickle(f'{BASE_DIR}/data/processed/corpus.pkl')

In [11]:
data_dtm["doc"] = data_dtm["text_clean"].apply(normalize)
data_dtm["doc"]

newspaper
DiarioElPeruano    [conoce, actividades, virtuales, desarrollarán, aniversario, lima, transmitido, facebook, youtube, ministra, defensa, nuria, esparch, resalta, presencia, mujeres, misiones, paz, ops, alerta, variante, expande, latinoamérica, crisis, política, amenaza, futuro, imperio, económico, trump, debemos, ...
ExpresoPeru        [giulliana, loza, keiko, fujimori, realizando, visitas, norte, país, debida, autorización, judicial, médicos, intensivistas, sostienen, lima, metropolitana, debió, calificada, región, nivel, alerta, alto, partido, aston, villa, vs, everton, aplazado, casos, coronavirus, policía, judicial, buscar...
Gestionpe          [depósitos, dólares, soles, convendrá, año, sinovac, defiende, vacuna, datos, decepcionantes, ensayo, brasil, línea, metro, lima, establece, horario, ampliación, toque, queda, alemania, temen, variante, británica, covid, multiplicar, diez, incidencia, geely, foxconn, crean, alianza, fabricar, ve...
diariocorreo       [san, borja, m

In [13]:
data_dtm

,text_clean,doc
newspaper,,
DiarioElPeruano,conoce las actividades virtuales que se desarrollarán por el aniversario de lima será transmitido por facebook youtube ministra de defensa nuria esparch resalta la presencia cada vez mayor de mujeres en misiones de paz ops alerta que la variante del se expande en latinoamérica crisis política...,"[conoce, actividades, virtuales, desarrollarán, aniversario, lima, transmitido, facebook, youtube, ministra, defensa, nuria, esparch, resalta, presencia, mujeres, misiones, paz, ops, alerta, variante, expande, latinoamérica, crisis, política, amenaza, futuro, imperio, económico, trump, debemos, ..."
ExpresoPeru,giulliana loza keiko fujimori está realizando visitas al norte del país con la debida autorización judicial médicos intensivistas sostienen que lima metropolitana debió ser calificada como una región con un nivel de alerta muy alto partido entre aston villa vs everton fue aplazado por casos de...,"[giulliana, loza, keiko, fujimori, realizando, visitas, norte, país, debida, autorización, judicial, médicos, intensivistas, sostienen, lima, metropolitana, debió, calificada, región, nivel, alerta, alto, partido, aston, villa, vs, everton, aplazado, casos, coronavirus, policía, judicial, buscar..."
Gestionpe,depósitos en dólares en soles qué convendrá más este año sinovac defiende su vacuna tras datos decepcionantes de ensayo en brasil línea del metro de lima establece nuevo horario ante ampliación del toque de queda en alemania temen que variante británica del covid pueda multiplicar por diez la...,"[depósitos, dólares, soles, convendrá, año, sinovac, defiende, vacuna, datos, decepcionantes, ensayo, brasil, línea, metro, lima, establece, horario, ampliación, toque, queda, alemania, temen, variante, británica, covid, multiplicar, diez, incidencia, geely, foxconn, crean, alianza, fabricar, ve..."
diariocorreo,san borja minsa lanzan farmacia con servicio delivery para población vulnerable médicos de essalud extirpan tumor de un kilo salvan niño arequipeño de años video hombre de años murió después de cobrar su pensión en un banco de huancayo jne declara infundada apelación del apra para inscripci...,"[san, borja, minsa, lanzan, farmacia, servicio, delivery, población, vulnerable, médicos, essalud, extirpan, tumor, kilo, salvan, niño, arequipeño, años, video, hombre, años, murió, cobrar, pensión, banco, huancayo, jne, declara, infundada, apelación, apra, inscripción, listas, parlamentarias, r..."
diarioojo,octogenario muere después de cobrar su pensión en un banco de junín luciana fuster celebra su cumpleaños en yate de marc anthony el invitado de honor video mujer acusada de asesinar su esposo él se apuñaló solo video gigi mitre enfrenta son tentación hay que ser un poquito más conscientes res...,"[octogenario, muere, cobrar, pensión, banco, junín, luciana, fuster, celebra, cumpleaños, yate, marc, anthony, invitado, honor, video, mujer, acusada, asesinar, esposo, apuñaló, video, gigi, mitre, enfrenta, tentación, poquito, conscientes, responsables, video, san, borja, minsa, lanzan, farmaci..."
elcomercio_peru,keiko fujimori me llena de ilusión energía haber empezado caminar para llegar todo nuestro país línea del metro de lima nuevo horario ante ampliación del toque de queda la policía los militares de eeuu se enfrentan un enemigos internos autos antiguos en las calles limeñas sporting cristal of...,"[keiko, fujimori, llena, ilusión, energía, empezado, caminar, llegar, país, línea, metro, lima, horario, ampliación, toque, queda, policía, militares, eeuu, enfrentan, enemigos, internos, autos, antiguos, calles, limeñas, sporting, cristal, oficializó, regreso, jesús, pretell, temporada, real, m..."
larazon_pe,la premier violeta bermúdez dijo que se empezará una campaña informativa sobre la importancia de la vacunación ya que muchas personas tienen temor no creen el exministro de salud lamentó que la titular del minsa no pueda inducir al presidente estrategias mucho más claras juez del primer juzg

In [62]:
from collections import Counter

frecuency_df = []

for i in range(10):
    df = pd.DataFrame.from_dict(Counter(data_dtm.doc.loc[i]), orient='index', columns=[data_dtm.newspaper.loc[i]], dtype=np.int64).T
    frecuency_df.append(df)

frecuency_df[1]

,giulliana,loza,keiko,fujimori,realizando,visitas,norte,país,debida,autorización,...,estrictamente,esté,desactivación,terna,stephanie,frappart,vandálicos,american,airlines,ensaya
ExpresoPeru,2,2,8,9,1,1,11,52,1,4,...,1,1,1,1,1,1,1,1,1,1


In [66]:
dtm = pd.concat(frecuency_df, axis=1).fillna(0)
dtm

,conoce,actividades,virtuales,desarrollarán,aniversario,lima,transmitido,facebook,youtube,ministra,...,plata,reniec,vencido,usarse,junio,raparle,enterarse,nieta,electrónica,lograron
DiarioElPeruano,18.0,26.0,7.0,2.0,9.0,95.0,1.0,3.0,2.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ExpresoPeru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gestionpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diariocorreo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
diarioojo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elcomercio_peru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
larazon_pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
larepublica_pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
peru21noticias,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tromepe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [67]:
dtm.to_pickle(f'{BASE_DIR}/data/interim/dtm.pkl')